In [1]:
from azureml.core import Workspace, Dataset
import urllib.request
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice

subscription_id = '04b1ca0e-558c-48d0-ba28-fee796a82076'
resource_group = 'analytics2040c59b'
workspace_name = 'VitableML'

workspace = Workspace(subscription_id, resource_group, workspace_name)

In [103]:
## REGISTER AND CONFIGURE MODEL

model = Model.register(workspace, model_name="quizparameterrecommender", model_path="./source_files/recommender_algorithm.py")

env = Environment.from_pip_requirements(name="quizparameterrecommender", file_path = "./requirements.txt")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_files",
    entry_script="./recommender_algorithm.py",
)

deployment_config = LocalWebservice.deploy_configuration(port=6799)


Registering model quizparameterrecommender


In [104]:
## DEPLOY

service = Model.deploy(
    workspace,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry mlresgistry0b7b758c.azurecr.io
Logging into Docker registry mlresgistry0b7b758c.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM mlresgistry0b7b758c.azurecr.io/azureml/azureml_3953a899d851c6104cb2ca06b98ab928
 ---> af1662684319
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 5415c223cbdd
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjA0YjFjYTBlLTU1OGMtNDhkMC1iYTI4LWZlZTc5NmE4MjA3NiIsInJlc291cmNlR3JvdXBOYW1lIjoiYW5hbHl0aWNzMjA0MGM1OWIiLCJhY2NvdW50TmFtZSI6InZpdGFibGVtbCIsIndvcmtzcGFjZUlkIjoiM2YzYzdkNzQtZmIzNy00NGU1LWFmNmUtNjRkY2UxMDhmNDIwIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 225e374bd6f0
 ---> 8807902542b3
Step 4/5 : RUN mv '/var/azureml-app/tmp_0lzm7dz.py' /var/azureml-app/main.py
 ---> Running in 7f59a79f7a80
 ---> cbf3ef50a58d
Step 5/5 : CMD

In [107]:
## TRY IT OUT

import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6799")
headers = {"Content-Type": "application/json"}
quiz_payload = {
  "version": "prod_1",
  "payload": {
    "arguments": {
      "q_age": 65,
      "q_alcoholic_drink_a_night": "daily",
      "q_allergy": "none",
      "q_antibiotics": "no",
      "q_bones": "no",
      "q_brain_fog": "never",
      "q_condition": "no",
      "q_constipated": "no",
      "q_currently_take": "no",
      "q_dairy":	"1_2_per_week",
      "q_describe_diets": "any",
      "q_diarrhea": "no",	
      "q_digestive_health": "bloating",
      "q_eat_protein": "",
      "q_energy_level": 1,
      "q_energy_slumps_experience": "daily",
      "q_exercise_before_bed": "",	
      "q_exercise_days": "1_day",
      "q_exercise_intensity_rating": 1,
      "q_exercise_type": "walking",
      "q_feel_stressed": "daily",
      "q_feel_unwell": "more_than_4",
      "q_feeling_sluggish": "daily",
      "q_fish": "rarely",
      "q_fitness_focus": "",
      "q_focusing": "never",
      "q_fruit_vegetables": "1_2_serves",
      "q_gallstone": "no",
      "q_gender": "female",
      "q_hair": "none",
      "q_heart_concern": "none",
      "q_history_of_heart_problems": "yes",
      "q_improve_area": "energy,digestion,sleep,stress,immunity,hair,skin,joints,brain,bones,heart",
      "q_iron": "no",
      "q_joints": "joint_pain",
      "q_meat": "3_per_week",
      "q_medications_taking": "other",
      "q_memory": "yes",
      "q_muscle_cramps": "",
      "q_natal": "no",
      "q_often_get_cold": "no",
      "restriction": "none",
      "q_skin": "ageing,dryness,elasticity,acne",
      "q_sleep_screen": "",
      "q_smoke": "no",
      "q_stress_level": 5,
      "q_sugar": "yes",
      "q_sunshine": "rarely",
      "q_supplement_perception": "",
      "q_take_prescription": "yes",
      "q_taken_in_the_past": "occasional",
      "q_trying_form_baby": "no",
      "q_urinary_tract_health": "no",
      "immunity_goal": 1,
      "energy_goal": 1,
      "digestion_goal": 1,
      "sleep_goal": 1,
      "skin_goal": 1,
      "brain_goal": 1,
      "heart_goal": 1,
      "hair_goal": 1,
      "joints_goal": 1,
      "bones_goal": 1,
      "stress_goal": 1,
      "number_goals": 11      
    },
    "parameterBounds": {}
  }
}    

quiz_payload = json.dumps(quiz_payload)
response = requests.post(uri, data=quiz_payload, headers=headers)
print(response)
print(response.json())

<Response [200]>
{'recommendedParameters': {'pills_cap': 10, 'bulk_cap': 2, 'basket_value_cap': 100}}


In [92]:
service.get_logs()

'2021-07-01T02:19:16,174420099+00:00 - gunicorn/run \nFile not found: /var/azureml-app/.\nStarting HTTP server\n2021-07-01T02:19:16,177232244+00:00 - rsyslog/run \n2021-07-01T02:19:16,177720752+00:00 - nginx/run \n2021-07-01T02:19:16,181195007+00:00 - iot-server/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-07-01T02:19:16,248525683+00:00 - iot-server/finish 1 0\n2021-07-01T02:19:16,249625101+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (11)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 41\nSPARK_HOME not set. Skipping PySpark Initialization.\nInitializing logger\n2021-07-01 02:19:16,787 | root | INFO | Starting up app insights client\nlogging socket was found. logging is available.\nlogging socket was found. logging is available.\n2021-07-01 02:19:16,787 | root | INFO | Starting up request id generator\n2021-07-01 02:19:16,788 | root | INFO

In [109]:
## DEPLOY TO CLOUD
model_version = '1p1'

tags = {
    'source': 'vitable',
    'production': False,
    'version': model_version
}

from azureml.core.webservice import AciWebservice

aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5, 
    memory_gb = 1, 
    tags=tags, 
    auth_enabled=True)

service = Model.deploy(workspace, 
    "quizparameterrecommender", 
    [model], 
    dummy_inference_config, 
    aci_deployment_config)

service.wait_for_deployment(show_output=True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-01 03:48:09+00:00 Creating Container Registry if not exists.
2021-07-01 03:48:09+00:00 Registering the environment.
2021-07-01 03:48:09+00:00 Use the existing image.
2021-07-01 03:48:09+00:00 Generating deployment configuration.
2021-07-01 03:48:10+00:00 Submitting deployment to compute..
2021-07-01 03:48:17+00:00 Checking the status of deployment quizparameterrecommender..
2021-07-01 03:49:50+00:00 Checking the status of inference endpoint quizparameterrecommender.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-07-01T03:49:39,495231700+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-07-01T03:49:39,511169900+00:00 - nginx/run 
2021-07-01T03:49:39,521148000+00:00 - iot-server/run 
2021-07-01T03:49:39,558972900+00:00 - rsyslog/run